In [ ]:
import pickle

import nengo
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import nengo_dl

tf.autograph.set_verbosity(0)

In [ ]:
# load the dataset 
root_dir = 'traffic-signs-preprocessed/'

train = pickle.load(open(root_dir + 'train.pickle', 'rb'))
test = pickle.load(open(root_dir + 'test.pickle', 'rb'))
labels = pickle.load(open(root_dir + 'labels.pickle', 'rb'))

NUM_CLASSES = 43

train_images = train['features'][:, :, :, 0]
train_labels = train['labels']
train_images = train_images.reshape((train_images.shape[0], -1))

test_images = test['features'][:, :, :, 0]
test_labels = test['labels']
test_images = test_images.reshape((test_images.shape[0], -1))

print(train_images.shape)
print(test_images.shape)

# add single timestep to training data
train_images = train_images[:, None, :]
train_labels = train_labels[:, None, None]

# when testing our network with spiking neurons we will need to run it
# over time, so we repeat the input/target data for a number of
# timesteps.
n_steps = 25
test_images = np.tile(test_images[:, None, :], (1, n_steps, 1))
test_labels = np.tile(test_labels[:, None, None], (1, n_steps, 1))

In [ ]:
# Show some examples
plt.figure(figsize=(12, 4))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.imshow(np.reshape(train_images[i], (28, 28)), cmap="gray")
    plt.axis("off")
    plt.title(str(train_labels[i]))

In [ ]:
# Model definition
# CNN with LIF neurons
with nengo.Network(seed=0) as net:
    # set some default parameters for the neurons that will make
    # the training progress more smoothly
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([100])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None
    neuron_type = nengo.LIF(amplitude=0.01)

    # this is an optimization to improve the training speed,
    # since we won't require stateful behaviour in this example
    nengo_dl.configure_settings(stateful=False)

    # the input node that will be used to feed in input images
    inp = nengo.Node(np.zeros(32 * 32))

    # add the first convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(filters=32, kernel_size=3))(
        inp, shape_in=(32, 32, 1)
    )
    x = nengo_dl.Layer(neuron_type)(x)

    # add the second convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(filters=64, strides=2, kernel_size=3))(
        x, shape_in=(30, 30, 32)
    )
    x = nengo_dl.Layer(neuron_type)(x)

    # add the third convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(filters=128, strides=2, kernel_size=3))(
        x, shape_in=(14, 14, 64)
    )
    x = nengo_dl.Layer(neuron_type)(x)

    # linear readout
    out = nengo_dl.Layer(tf.keras.layers.Dense(units=NUM_CLASSES))(x)

    # we'll create two different output probes, one with a filter
    # (for when we're simulating the network over time and
    # accumulating spikes), and one without (for when we're
    # training the network using a rate-based approximation)
    out_p = nengo.Probe(out, label="out_p")
    out_p_filt = nengo.Probe(out, synapse=0.1, label="out_p_filt")

In [ ]:
# define the accuracy metric and minibatch size

def classification_accuracy(y_true, y_pred):
    return tf.metrics.sparse_categorical_accuracy(y_true[:, -1], y_pred[:, -1])

minibatch_size = 128
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size)

In [7]:
# run training or load pre-trained parameter
do_training = False

if do_training:
    sim.compile(
        optimizer=tf.optimizers.RMSprop(0.001),
        loss={out_p: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
    )
    sim.fit(train_images, {out_p: train_labels}, epochs=15)
    sim.save_params("./tafficSign_params")
else:
    sim.load_params("./trafficSign_params")

271/271 [==============================] - 142s 505ms/step - loss: 1.3840 - out_p_loss: 1.3840
Epoch 2/15
271/271 [==============================] - 145s 537ms/step - loss: 0.2490 - out_p_loss: 0.2490
Epoch 3/15
132/271 [=============>................] - ETA: 1:13 - loss: 0.1300 - out_p_loss: 0.1300

In [ ]:
# Evaluate the model
sim.compile(loss={out_p_filt: classification_accuracy})

print(
    "Accuracy after training:",
    sim.evaluate(test_images, {out_p_filt: test_labels}, verbose=0)["loss"],
)

In [ ]:
# Show some example predictions
data = sim.predict(test_images[:minibatch_size])

for i in range(5):
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(test_images[i, 0].reshape((28, 28)), cmap="gray")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.plot(tf.nn.softmax(data[out_p_filt][i]))
    plt.legend([str(i) for i in range(10)], loc="upper left")
    plt.xlabel("timesteps")
    plt.ylabel("probability")
    plt.tight_layout()

In [ ]:
#sim.close()